<h1><center>Loan Defaulter Prediction</center></h1>

<h3>We have dataset of shape (39717,111) where 39717 is total row and 111 are total features. Here we have to pre-processed data because there are many redundant for eg- many column have all nan value some column have all same data in all row. When our pre-processing finished our dataset reduce to (39717, 31). We have to analyze our pre-processing data first seperate label and features And divide whole dataset into two parts testing and for training. Here we have .33 percent of whole dataset as testing file and remaining as training part. Then we tried different model for training and we see that for these best is <bold>Logistic Regression</bold> .</h3>
<h3>Here our label consist of <bold> Charged off, Current , Fully Paid</bold>. We can consider Current and Fully Paid as Non Defaulter and Charged off as Defaulter</h3>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_excel('./data/loan.xlsx')

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
detail = pd.read_excel('./data/Data_Dictionary.xlsx')

In [ ]:
import seaborn as sns

In [ ]:
fig = plt.figure(figsize=(18,6))
miss_train = pd.DataFrame((data.isnull().sum())*100/data.shape[0]).reset_index()
miss_train["type"] = "train"
missing = pd.concat([miss_train],axis=0)
ax = sns.pointplot("index",0,data=missing,hue="type")
plt.xticks(rotation =90,fontsize =7)
plt.title("Percentage of Missing values in application train data")
plt.ylabel("PERCENTAGE")
plt.xlabel("COLUMNS")
ax.set_facecolor("k")
fig.set_facecolor("lightgrey")

In [ ]:
data.isnull().sum(axis = 0)

In [ ]:
sum(data.isnull().sum(axis = 0) == 0)

In [ ]:
data.shape

In [ ]:
df = data.dropna( axis = 1, how ='all')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
plt.plot(df['loan_amnt'])
print('Max of loan amount : {} Min of loan amount :{}'.format(max(df['loan_amnt']),min(df['loan_amnt'])))

In [ ]:
plt.plot(df['funded_amnt'])
print('Max of funded loan amount : {} Min of funded loan amount :{}'.format(max(df['funded_amnt']),min(df['funded_amnt'])))

In [ ]:
sum(df['funded_amnt'] - df['loan_amnt'] != 0)          #number of funded amount that is difference from loan amount

In [ ]:
plt.plot(df['funded_amnt_inv'])
print('Max of funded amount inv : {} Min of funded amount inv :{}'.format(max(df['funded_amnt_inv']),min(df['funded_amnt_inv'])))

In [ ]:
df.term.unique()

In [ ]:
import warnings

In [ ]:
df['term'] = df['term'].map({' 36 months' : 36, ' 60 months': 60})

In [ ]:
plt.plot(df['int_rate'])
print('Max of interest rate : {} Min of interest rate :{}'.format(max(df['int_rate']),min(df['int_rate'])))

In [ ]:
plt.plot(df['installment'] * df['term'] - df['loan_amnt'], 'r')
print('Max of installment : {} Min of installment :{}'.format(max(df['installment']),min(df['installment'])))

In [ ]:
df.isnull().sum(axis = 0)

In [ ]:
lis = [u'id', u'member_id', u'emp_title', u'desc', u'url', u'zip_code', u'mths_since_last_delinq', u'mths_since_last_record', u'next_pymnt_d', u'addr_state', u'title', u'pub_rec_bankruptcies', u'tax_liens', u'policy_code', u'chargeoff_within_12_mths', u'collections_12_mths_ex_med', u'last_credit_pull_d', u'last_pymnt_d', u'pymnt_plan', u'initial_list_status', u'application_type', u'acc_now_delinq', u'delinq_amnt', u'issue_d', u'earliest_cr_line']

In [ ]:
data_new  = df[[i for i in df.columns if i not in lis]]


In [ ]:
data_new.head()

In [ ]:
data_new.isna().sum()

In [ ]:
lim = []
for i in data_new.columns:
    if len(data_new[i].unique()) < 5 :
        lim.append(i)

In [ ]:
data_new.info()

In [ ]:
data_new['delinq_2yrs'].unique()

In [ ]:
data_new.info()

In [ ]:
Y = data_new['loan_status']
data_new = data_new.drop(['loan_status'], axis = 1)

In [ ]:
obj_dtypes = [i for i in data_new.select_dtypes(include=np.object).columns ]
num_dtypes = [i for i in data_new.select_dtypes(include = np.number).columns]

In [ ]:
li_imp  = data_new.columns

In [ ]:
obj_dtypes

In [ ]:
num_dtypes

In [ ]:
for i in li_imp :
    data_new[i]  = data_new[i].fillna(data_new[i].mode()[0])

In [ ]:
data_new.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for i in obj_dtypes:
    #print(le.fit_transform(data_new[i]))
    data_new[i] = le.fit_transform(data_new[i])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_new, Y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report
def class_report(model,X_text,y_test):
    y_pred = model.predict(X_test)
    print(classification_report(y_test,y_pred))

In [ ]:
class_report(clf,X_test, y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
class_report(rfc,X_test, y_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
class_report(dtc,X_test, y_test)

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
class_report(gnb,X_test, y_test)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)
class_report(gbc,X_test, y_test)